<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/miniproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/frame/trainnorm.zip -d /train
!unzip /content/drive/MyDrive/frame/trainweap.zip -d /train

!unzip /content/drive/MyDrive/frame/testnorm.zip -d /test
!unzip /content/drive/MyDrive/frame/testweap.zip -d /test

In [ ]:
# 'https://drive.google.com/file/d/1-9AGfp6yJEpxWFFNlHoO4zEQmay_IiVQ/view?usp=sharing'
# 'https://drive.google.com/file/d/1-D0y52EAHNCMUd57UIrDkWI1ah64YFyI/view?usp=sharing'
# 'https://drive.google.com/file/d/1lXuUbBigytTraot3hlYQ4oqT8-JPmdRZ/view?usp=sharing'
# 'https://drive.google.com/file/d/1q1qIgz81KfWGFFkGuWkzGYH_zz8IPMun/view?usp=sharing'

!pip install --upgrade --no-cache-dir gdown

!gdown 1-9AGfp6yJEpxWFFNlHoO4zEQmay_IiVQ -O testnorm.zip
!gdown 1-D0y52EAHNCMUd57UIrDkWI1ah64YFyI -O testweap.zip
!gdown 1lXuUbBigytTraot3hlYQ4oqT8-JPmdRZ -O trainnorm.zip
!gdown 1q1qIgz81KfWGFFkGuWkzGYH_zz8IPMun -O trainweap.zip

!unzip /content/trainnorm.zip -d /train
!unzip /content/trainweap.zip -d /train
!unzip /content/testnorm.zip -d /test
!unzip /content/testweap.zip -d /test


###fast

In [1]:
# @title data
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torchvision
# import torchvision.transforms as transforms
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
# https://github.com/python-engineer/pytorchTutorial/blob/master/14_cnn.py

# dataset has PILImage images of range [0, 1], transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224)), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# transform = transforms.Compose(transforms.ToTensor())

train_dir='/train/content/'
test_dir='/test/content/'

# train_data = datasets.ImageFolder(train_dir, transform=transform)
# test_data = datasets.ImageFolder(test_dir, transform=transform)

# batch_size = 64 # 4

# # train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
# # test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# def imshow(img):
#     img = img / 2 + 0.5  # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# dataiter = iter(train_loader) # get some random training images
# images, labels = next(dataiter)
# # imshow(torchvision.utils.make_grid(images))

In [ ]:
# @title simplifi
# https://github.com/JayPatwardhan/ResNet-PyTorch/blob/master/ResNet/ResNet.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels),
        )
        self.i_downsample = i_downsample

    def forward(self, x):
        identity = x.clone()
        x = self.conv(x)
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        x += identity
        x = nn.ReLU()(x)
        return x


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels), nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels), nn.ReLU(),
            nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels*self.expansion),
        )
        self.i_downsample = i_downsample
        
    def forward(self, x):
        identity = x.clone()
        x = self.conv(x)
        if self.i_downsample is not None: #downsample if needed
            identity = self.i_downsample(identity)
        x += identity #add identity
        x = nn.ReLU()(x)
        return x


class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        # https://github.com/akamaster/pytorch_resnet_cifar10/blob/master/resnet.py
        # num_blocks=[3,3,3] aka layer_list
        # plane_list=[64,128,256]
        plane_list=[16,32,64] #og
        self.in_channels = plane_list[0]
        self.conv = nn.Sequential(
            nn.Conv2d(3, plane_list[0], kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(plane_list[0]), nn.ReLU(),
            self._make_layer(ResBlock, layer_list[0], plane_list[0], stride=1),
            self._make_layer(ResBlock, layer_list[1], plane_list[1], stride=2),
            self._make_layer(ResBlock, layer_list[2], plane_list[2], stride=2),
            nn.AdaptiveAvgPool2d((1,1)),
        )
        # self.fc = nn.Linear(plane_list[2]*ResBlock.expansion, num_classes)
        self.cc = nn.Conv2d(plane_list[2]*ResBlock.expansion, num_classes, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.conv(x)
        # print("forward x",x.shape)
        # x = x.reshape(x.shape[0], -1)
        # x = self.fc(x)
        x = self.cc(x)
        x = torch.squeeze(x)
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
        return nn.Sequential(*layers)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = ResNet(Bottleneck, [3,4,6,3], num_classes=10, num_channels=3).to(device)
model = ResNet(Bottleneck, [3,3,3], num_classes=2, num_channels=3).to(device)
# print(model)

loss_list=[]

In [ ]:
# @title convnet

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         plane_list=[64,128,256,512]
#         self.conv = nn.Sequential( # nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
#             nn.Conv2d(3, plane_list[0], 3, 1, 1), nn.BatchNorm2d(plane_list[0]), nn.ReLU(),# nn.MaxPool2d(2, 2),
#             nn.Conv2d(plane_list[0], plane_list[1], 5, 1, 2), nn.BatchNorm2d(plane_list[1]), nn.ReLU(), nn.MaxPool2d(2, 2),
#             nn.Conv2d(plane_list[1], plane_list[2], 7, 1, 3), nn.BatchNorm2d(plane_list[2]), nn.ReLU(), nn.MaxPool2d(2, 2),
#         )
#         self.linear = nn.Sequential(
#             # nn.Linear(16 * 8 * 8, 256), nn.ReLU(),
#             nn.Linear(plane_list[2] * 8 * 8, 256), nn.ReLU(),
#             # nn.Linear(plane_list[2]//16, 256), nn.ReLU(),
#             nn.Linear(256, 64), nn.ReLU(),
#             nn.Linear(64, 10),
#         )

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        plane_list=[64,128,256,512]
        self.conv = nn.Sequential( # nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.Conv2d(3, plane_list[0], 3, 1, 1), nn.BatchNorm2d(plane_list[0]), nn.ReLU(), #nn.MaxPool2d(2, 2),
            nn.Conv2d(plane_list[0], plane_list[1], 3, 1, 1), nn.BatchNorm2d(plane_list[1]), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(plane_list[1], plane_list[2], 3, 1, 1), nn.BatchNorm2d(plane_list[2]), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Conv2d(plane_list[2],64, 1, 1, 0), #nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.Conv2d(64, 10, 1, 1, 0),
        )

    def forward(self, x):
        x = self.conv(x)
        # print("forward x",x.shape)
        # x = x.view(-1, 16 * 5 * 5)
        # x = nn.Flatten()(x)
        # x = self.linear(x)
        x = torch.squeeze(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ConvNet().to(device)
# print(model)

loss_list=[]

In [ ]:
# @title pretrained
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, models, transforms
# https://pytorch.org/vision/0.12/models.html#id10
model = models.resnet152(pretrained=True) # 18 34 50 101 152
num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 2),
    nn.Softmax(),
    )

c=0
for param in model.parameters():
    # print(c, param.shape)
    if c>=460: #101(312) 152(466)
        param.requires_grad = True
    else:
        param.requires_grad = False
    c+=1

# for param in model.parameters():
#     print(param.requires_grad,param.shape)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# print(model)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
c=0
for param in model.parameters():
    param.requires_grad = False
    # print(param.shape)
    c+=1
    if c>=0: #312
        param.requires_grad = True

In [ ]:
X = torch.rand(64, 3, 32, 32, device=device)
logits = model(X)
print(logits.shape)
# print(logits[0])
# print(logits[0].argmax(1))
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")


torch.Size([64, 2])


In [ ]:
# @title train test function

scaler = torch.cuda.amp.GradScaler()
# for data, label in data_iter:
#    optimizer.zero_grad()
#    # Casts operations to mixed precision
#    with torch.cuda.amp.autocast():
#       loss = model(data)
#    scaler.scale(loss).backward()
#    scaler.step(optimizer)
#    scaler.update()

# def strain(dataloader, model, loss_fn, optimizer):
def strain(dataloader, model, loss_fn, optimizer, scheduler=None, verbose=True):
    size = len(dataloader.dataset)
    model.train()
    # lr_list, loss_list = [], []
    for batch, (x, y) in enumerate(dataloader):
        sx, sy = x.to(device), y.to(device)
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast():
            pred = model(sx)
            loss = loss_fn(pred, sy)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        if scheduler is not None:
            # lr_list.append(optimizer.param_groups[0]["lr"])
            scheduler.step()
            # loss_list.append(loss.item())

        # loss.backward()
        # optimizer.step()
        if (batch) % (size//(10* len(x))) == 0:
            loss, current = loss.item(), batch * len(x)
            if verbose: print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return #lr_list, loss_list

# def train(dataloader, model, loss_fn, optimizer):
def train(dataloader, model, loss_fn, optimizer, scheduler=None, verbose=True):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
    # for batch, ((x,y), labels) in enumerate(dataloader):
        sx, sy = x.to(device), y.to(device)
        # print("sx sy",sx.shape,sy.shape)
        pred = model(sx)
        loss = loss_fn(pred, sy)
        # loss = model.loss(sx,sy)
        optimizer.zero_grad() # reset gradients of model parameters, to prevent double-counting
        loss.backward() # Backpropagate gradients
        optimizer.step() # adjust the parameters by the gradients
        if scheduler is not None: scheduler.step()
        if (batch) % (size//(10* len(x))) == 0:
            loss, current = loss.item(), batch * len(x)
            loss_list.append(loss)
            if verbose: print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# def test(dataloader, model, loss_fn):
def test(dataloader, model, loss_fn, verbose=True):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            x, y = X.to(device), y.to(device)
            pred = model(x)
            loss = loss_fn(pred, y)
            # predicted, actual = classes[pred[0].argmax(0)], classes[y]
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    # if verbose: print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    if verbose: print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(lr_list,acc_list)
# plt.plot(acc_list)
plt.plot(lr_list, loss_list)
# plt.plot(lr_list[:-40], loss_list[:-40])
plt.xscale('log')
plt.show()
plt.close()


In [ ]:
#  time epoch
import time
start = time.time()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
strain(train_loader, model, loss_fn, optimizer, verbose=False)

end = time.time()
print("time: ",end - start)


In [ ]:
# @title wwwwwwwww
import time
start = time.time()
lr_lst, loss_lst=[],[]
loss_fn = nn.CrossEntropyLoss()

# model = ResNet(Bottleneck, [3,3,3], num_classes=10, num_channels=3).to(device)
batch_size = 128 #64 # 4 512

train_data = datasets.ImageFolder(train_dir, transform=transform)
test_data = datasets.ImageFolder(test_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
num_batches=int(np.ceil(len(train_data)/batch_size))
train_data=0;test_data=0

base_lr, max_lr = 1e-5, 1e-3 #0.5#
end_lr, start_lr = 0.0001,0.1

# print(num_batches)
epochs = 10 #5 20
# (1e-5/1e-1)=gamma**(num_batches*epochs)
# gamma = np.exp(np.log(1e-3/1e-1)/epochs) # for scheduler step every epoch
gamma = np.exp(np.log(end_lr/start_lr)/(num_batches*epochs)) # for scheduler step every optimizer step
# print(gamma)
# optimizer = torch.optim.SGD(model.parameters(), lr = 1e-1, momentum=0.9)
optimizer = torch.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)

div_factor = max_lr/base_lr
# div_factor = 25
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=num_iter, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=10000.0, three_phase=True,)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, epochs=epochs, steps_per_epoch=num_batches, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=10000.0, three_phase=True,)
# optimizer = torch.optim.AdamW(model.parameters(), lr=start_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # 0.75(20)-0.9(100)
total_steps=num_batches*epochs


for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    lr=optimizer.param_groups[0]["lr"]
    # lr_lst.append(lr)
    print(lr)
    # train(train_loader, model, loss_fn, optimizer)
    # lr_ls, loss_ls = strain(train_loader, model, loss_fn, optimizer, scheduler)
    strain(train_loader, model, loss_fn, optimizer, scheduler)
    test(test_loader, model, loss_fn)
    # scheduler.step()
    # lr_lst.append(lr_ls); loss_lst.append(loss_ls)
print("Done!")

end = time.time()
print("time: ",end - start)

torch.save(model.state_dict(), "model.pth")

# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

Epoch 1
-------------------------------
1.0000000000000026e-05


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 0.669660  [    0/ 8329]


In [ ]:

pth='/content/drive/MyDrive/frame/res152.pth'

# torch.save(model.state_dict(), pth)
# model.load_state_dict(torch.load("model.pth"))


In [ ]:
test(test_loader, model, loss_fn)

Accuracy: 61.0%, Avg loss: 2.049678


0.6104120404638539